<div align='center' ><font size='70'>Deque</font></div>

<img align="right" src="./src/deque.png" width="50%" border="0">

## 双端队列 **deque**

* [How to pronounce deque?](https://cplusplus.com/reference/deque/deque/) **"deck"**

* 此队列非彼队列，不是**std::queue**

* 存储特性
    * 可以采用**operator[]**访问，**连续？**
    * 与**vector**区别？
    * 常数时间**首尾**插入删除元素

* 迭代器
    * 序列容器中最复杂的一种
    * 对外模拟**连续性**
    * **失效**？[**[deque.cpp]**](./code/deque.cpp)


<img align="right" src="./src/deque-map.png" width="52%" border="0">

## **deque** 基本实现

* **deque** 由一段一段的**定量连续**空间构成

* 中央控制器**map**，其实是一个二级指针**T\*\***

* 每个**map**节点指向一段**连续**空间（**buffer**）

* **buffer**空间是连续的（部分连续）

* **虚假**的连续性由**迭代器**维护

## **deque** 的迭代器

* 迭代器类型为 **Random Access Iterator**

* **BufferSize**默认是**0**，表示**buffer**大小由**元素大小**决定，最大为**512 bytes**

```c++
template <typename T, typename Ref, typename Ptr, size_t BufferSize>
struct __deque_iterator {
    typedef __deque_iterator<T, T&, T*, BufferSize> iterator;
    typedef __deque_iterator<T, const T&, const T*, BufferSize> const_iterator;
    static size_t buffer_size() { return __deque_buf_size(BufferSize, sizeof(T)); }
    typedef random_access_iterator_tag iterator_category;
    typedef T value_type;
    typedef Ptr pointer;
    typedef Ref reference;
    typedef size_t size_type;
    typedef ptrdiff_t difference_type;
    typedef T** map_pointer;
    typedef __deque_iterator self;

    T* cur;
    T* first;
    T* last;
    map_pointer node;
};
```

<img align="right" src="./src/deque_iterator.png" width="52%" border="0">

## **deque** 的[迭代器](../src/stl_deque.h)

* **first**和**last**分别指向当前**buffer**的头和尾

* **cur**指向当前**buffer**中的某个元素

* **node**指向**map**中的某个节点

* 重载运算符

> **operator++**, **operator--**, **operator+**, **operator-**, **operator+=**, **operator-=**, **operator[]**

## **deque**的[数据结构](../src/stl_deque.h)

* **start**，**finish**分别指向**map**中的第一个和最后一个节点

* **map**是一个**T\*\***类型的指针数组

* **map_size**存储**map**大小

```c++
template<typename Tp, typename Alloc>
class deque_base {
public:
	typedef deque_iterator<Tp, Tp&, Tp*>             iterator;
	typedef deque_iterator<Tp, const Tp&, const Tp*> const_iterator;
	typedef Alloc allocator_type;
// ...
protected:
	Tp** m_map;
	size_t m_map_size;
	iterator m_start;
	iterator m_finish;
};

// deque
template<typename Tp, typename Alloc=alloc>
class deque :protected deque_base<Tp, Alloc> {

private:
	typedef deque_base<Tp, Alloc> Base;
public:
	typedef Tp        value_type;
	typedef Tp*       pointer;
	typedef const Tp* const_pointer;
	typedef Tp&       reference;
	typedef const Tp& const_reference;
	typedef size_t    size_type;
	typedef ptrdiff_t difference_type;
// ...
};
```


## **deque** [内存管理](../src/stl_deque.h)

* 如何做到双端插入都是常数时间

* 数据总是放中间，**前后**预留相当的空间

```c++
enum { INITIAL_MAP_SIZE = 8 };
void initialize_map(size_t num_elements) {
	size_t num_nodes = num_elements / buf_size(sizeof(Tp)) + 1;
	m_map_size = max((size_t)INITIAL_MAP_SIZE, num_nodes + 2);
	// allocate memory
	m_map = allocate_map(m_map_size);
	Tp** start = m_map + (m_map_size - num_nodes) / 2;
	Tp** finish = start + num_nodes;
	create_node(start, finish);
	m_start.set_node(start);
	m_finish.set_node(finish - 1);
	m_start.m_cur = m_start.m_first;
	m_finish.m_cur = m_finish.m_first + num_elements % buf_size(sizeof(Tp));
}
void create_node(Tp** start, Tp** finish) {
	Tp** cur = nullptr;
	for (cur = start; cur < finish; ++cur) {
		*cur = allocate_node();
	}
}
```




<img align="right" src="./src/deque_node_map.png" width="52%" border="0">

## **push_back** 实现

* **push_front**实现类似

* **deque**同样支持任意位置**插入**和**删除**

* 同样面临**copy**和**move**元素问题

* 与**vector**类似

* 是**stack**和**queue**默认容器（**Adaptor**）

```c++
void push_back(const value_type& value) {
    if (m_finish.m_cur != m_finish.m_last - 1) {
        construct(m_finish.m_cur, value);
        ++m_finish.m_cur;
    }
    else {
        push_back_aux(value);
    }
}
// auxillary function
void push_back_aux(const value_type& value) {
    value_type value_copy = value;
    reserve_map_at_back();
    *(m_finish.m_node + 1) = allocate_node();
    construct(m_finish.m_cur, value_copy);
    m_finish.set_node(m_finish.m_node + 1);
    m_finish.m_cur = m_finish.m_first;
}
```

<div align='center' ><font size='70'>Adaptor</font></div>

## **stack**和**queue**的实现

* **stack**
    <img align="right" src="./src/stack.png" width="52%" border="0">
    * **First In Last Out FILO**
    * 没有迭代器

* **queue**
    <img align="right" src="./src/queue.png" width="52%" border="0">
    * **First In First Out FIFO**
    * 没有迭代器
    

* 二者默认的容器都是**deque**

* **Review** 其他容器可以吗？

* 源代码**十分**简短，~~可以自己动手尝试~~

## 优先队列 **priority queue**

* 和**queue**区别？

* **queue**是**FIFO**，**priority queue**是**优先级**高的先出

## 堆 **heap**
<img align="right" src="./src/complete-binary-tree.png" width="42%" border="0">

* 与**Operating System**中的**heap**不是一个概念

* **heap**是一种**完全二叉树**，**最大堆**和**最小堆**

* 树一定要使用指针结构吗？

* [**heap**](../src/stl_heap.h)更像是**算法**

## **priority queue**

* 默认容器是**vector**

* 不提供**迭代器**

* 可以自定义比较函数 **Functor**

* 源码简短，~~可以自己动手尝试~~

* **max-heap** **less<int>**，**min-heap** **greater<int>**

* 似乎和**sort**是相反的？

## 单向链表 **slist**

* 很少采用，**list**更常用 **非标准**

* ~~分析略~~


<div align='center' ><font size='70'>End</font></div>